# 상권,업종별 결측치 비율 계산

In [1]:
# 필요한 라이브러리 불러오기

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## 1. 데이터 불러오기

In [2]:
# 데이터 경로 설정
path = './통합_점포_추정매출_최종.csv'

# 2015-1분기 ~ 2020-2분기까지의 모든 상권 및 모든 서비스업종에 관한 데이터
df = pd.read_csv(path, encoding = 'cp949').drop('Unnamed: 0', axis = 1)

In [3]:
# 결과 확인

df.head()

,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2015,1,1001490,길동복조리시장,CS300011,일반의류,33,33,0,0,...,643.0,4389.0,11072.0,97.0,1205.0,2267.0,4759.0,4584.0,2549.0,33.0
1,2015,1,1001046,서울 관악구 신림역_4,CS300001,슈퍼마켓,18,18,0,0,...,48014.0,79772.0,108601.0,1293.0,68332.0,55144.0,26564.0,23568.0,13473.0,18.0
2,2015,1,1000609,개봉로11길,CS100001,한식음식점,9,9,0,0,...,664.0,1620.0,951.0,7.0,199.0,509.0,785.0,662.0,407.0,9.0
3,2015,1,1000910,도산대로81길,CS100004,양식음식점,9,9,22,2,...,1908.0,4735.0,6426.0,81.0,3347.0,4750.0,1784.0,929.0,276.0,10.0
4,2015,1,1000862,방배로18길,CS100002,중식음식점,1,2,0,0,...,75.0,1726.0,892.0,16.0,509.0,1021.0,731.0,220.0,121.0,2.0


In [4]:
# (행,열) 확인

df.shape

(1969593, 85)

## 2. 점포 수에 따른 결측치 비율 탐색

In [5]:
# 데이터를 점포수 기준으로 정렬

df = df.sort_values(by = '점포_수', ascending = False).reset_index(drop = True)
df.head()

,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2015,1,1001493,동대문패션타운 관광특구,CS300011,일반의류,13193,13212,3,402,...,354425.0,567352.0,1114039.0,30598.0,422376.0,526203.0,382602.0,225947.0,93673.0,13213.0
1,2015,2,1001493,동대문패션타운 관광특구,CS300011,일반의류,13070,13090,2,272,...,421275.0,716451.0,1437159.0,35954.0,544434.0,664762.0,471235.0,302821.0,134414.0,13122.0
2,2017,1,1001493,동대문패션타운 관광특구,CS300011,일반의류,13037,13058,4,528,...,237388.0,414821.0,856999.0,22206.0,302946.0,354492.0,288360.0,193524.0,110307.0,13020.0
3,2016,3,1001493,동대문패션타운 관광특구,CS300011,일반의류,13026,13048,5,591,...,347581.0,575207.0,1143397.0,30744.0,443646.0,503443.0,387618.0,237508.0,115654.0,13008.0
4,2016,4,1001493,동대문패션타운 관광특구,CS300011,일반의류,12998,13019,2,278,...,305704.0,538136.0,1032833.0,25769.0,371240.0,451570.0,354557.0,232406.0,135430.0,13028.0


In [6]:
# 1. 점포수가 가장 많은 상권과 업종을 선택하여 따로 추출
# 2. 결측치 비율을 계산 후 저장
# 3. 이미 찾은 상권과 업종 제거 후 반복

"""
import time
start = time.time()  # 시작 시간 저장

name_service_list = []
n_of_market_list = []
null_ratio_list = []
index_list = []


for i in range(len(df)):
    if i in index_list:
        continue
    else:
        row = df.iloc[i, :] # 행 추출
        n_of_market = row['점포_수']
        name, service = (row['상권_코드'], row['서비스_업종_코드'])
        n_of_market_list.append(n_of_market) # 점포 수 저장
        name_service_list.append((name, service)) # 상권과 서비스업종 저장
        name_df = df[df['상권_코드'] == name] # 같은 상권이면서
        service_df = name_df[name_df['서비스_업종_코드'] == service] # 같은 서비스 업종을 모두 추출
        n_of_all = service_df.shape[0] * service_df.shape[1] # 총 레코드 개수
        n_of_null = service_df.isnull().sum().sum() # 추출된 데이터의 결측치 총 개수
        null_ratio = n_of_null / n_of_all # 결측치 비율 계산
        null_ratio_list.append(null_ratio) # 결측치 비율 저장
        for j in list(service_df.index): # 인덱스를 저장하여 이미 조사한 행에 대해서는 다시 방문하지 않는다.
            index_list.append(j)
    
    
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

"""

time : 45938.85034632683


In [72]:
# 결과 저장

result_df = pd.DataFrame({'(상권코드, 업종코드)' : name_service_list, '점포수' : n_of_market_list, '결측비율' : null_ratio_list})

In [42]:
# 보기 편하게 정리 (상권코드와 업종코드를 분리하여 다시 저장)

result_df_new = result_df.copy()

result_df_new['상권코드'] = 0
result_df_new['업종코드'] = 0

for i in range(result_df_new.shape[0]):
    row = result_df_new.iloc[i]
    name = row['(상권코드, 업종코드)'][0]
    service = row['(상권코드, 업종코드)'][1]
    result_df_new.loc[i,'상권코드'] = name
    result_df_new.loc[i,'업종코드'] = service

In [43]:
# column 제거 및 순서 정리

result_df_new = result_df_new.drop('(상권코드, 업종코드)', axis = 1)
result_df_new = result_df_new[['상권코드', '업종코드', '점포수', '결측비율']]

In [44]:
# 결과 확인

result_df_new

,상권코드,업종코드,점포수,결측비율
0,1001493,CS300011,13193,0.000000
1,1001492,CS300011,3458,0.000000
2,1001207,CS300011,2389,0.000000
3,1001273,CS300011,2286,0.000000
4,1001279,CS300011,1857,0.000000
...,...,...,...,...
93242,1000567,CS300005,0,0.857143
93243,1000555,CS300005,0,0.857143
93244,1000553,CS300005,0,0.857143
93245,1000526,CS300005,0,0.857143


In [75]:
# 결과 저장

# result_df_new.to_csv('./결측치_비율계산_최종.csv', encoding = 'cp949')